In [1]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt
%matplotlib tk

In [2]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)
print('Scipy version ' + scipy.__version__)

Python version 3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Pandas version 1.0.1
Matplotlib version 3.1.3
Scipy version 1.4.1


In [6]:
# file path of csv file
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'

# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')
df

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,El 21,El 31,El 41,El 22,El 32,El 42,El 23,El 33,El 43
%t,,,,,,,,,
0.00,-2.84,-1.02,-7.05,-3.18,-3.41,-3.30,-2.27,-1.36,-2.95
0.04,-3.41,-3.30,-8.07,-2.39,-3.75,-2.27,-2.50,-2.27,-0.68
0.08,-4.77,-3.86,-5.80,-1.70,-2.73,-0.34,-2.95,-2.27,0.11
0.12,-5.00,-3.41,-4.32,-2.27,-2.84,0.45,-4.43,-3.64,-1.48
0.16,-3.30,-4.20,-4.09,-1.93,-4.55,1.14,-5.57,-3.52,-1.82
...,...,...,...,...,...,...,...,...,...
301799.80,0.68,0.68,0.23,-0.23,-1.25,3.07,-0.11,0.68,0.45
301799.84,-1.02,-1.14,0.91,0.23,-2.39,2.39,-2.50,1.82,-0.45
301799.88,-1.14,-3.41,0.91,-0.11,-3.41,2.05,-2.50,2.95,-1.36


In [63]:
########################################################################
###  SIGNAL ORIGINAL

##### SIGNAL TEMPOREL 

plt.figure()
df.iloc[:,1].plot(figsize=(100,10))
plt.xlabel('Time [ms]')
plt.ylabel('Amplitude [µV]')
plt.title('Original Signal')
plt.grid()
plt.show()

In [47]:
##### PERIODOGRAMME DU SIGNAL ORIGINAL

fs = 25000

plt.figure()
f_orig, Pxx_den_orig = signal.periodogram(df.iloc[:,1], fs)
plt.semilogy(f_orig,Pxx_den_orig)
plt.title('Periodogram of Original Signal')
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD')
plt.grid()
plt.show()

In [65]:
########################################################################
###  SIGNAL APRÈS PASSE-BANDE

##### SIGNAL TEMPOREL 

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y

# Sample rate and desired cutoff frequencies (in Hz).
fs = 25000.0
lowcut = 100.0
highcut = 2500.0


y = butter_bandpass_filter(df.iloc[:,1], lowcut, highcut, fs, order=6)

plt.figure(figsize = (100,20))
plt.plot(df.index, y, 'g-', linewidth=2, label='filtered data')
plt.title('Signal after Bandpass')
plt.xlabel('Time [ms]')
plt.ylabel('Amplitude [µV]')
plt.grid()
plt.axis('tight')
plt.show()


/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)


In [66]:
##### PERIODOGRAMME DU SIGNAL APRÈS PASSE-BANDE

fs = 25000

plt.figure()
f_BP, Pxx_den_BP = signal.periodogram(y, fs)
plt.semilogy(f_BP,Pxx_den_BP)
plt.title('Periodogram of signal after Bandpass')
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD')
plt.grid()
plt.show()

In [76]:
########################################################################
###  SIGNAL APRÈS PASSE-BANDE ET FILTRE WIENER

##### SIGNAL TEMPOREL 

y_wiener1 = signal.wiener(y,mysize=100)
plt.figure(figsize = (100,20))
plt.plot(df.index, y_wiener1, 'g-', linewidth=2, label='data after bandpass+wiener')
plt.title('Signal after Bandpass and Wiener')
plt.xlabel('Time [ms]')
plt.ylabel('Amplitude [µV]')
plt.grid()
plt.axis('tight')
plt.show()

In [77]:
##### PERIODOGRAMME DU SIGNAL APRÈS PASSE-BANDE ET WIENER

fs = 25000

plt.figure()
f_BPWiener, Pxx_den_BPWiener = signal.periodogram(y_wiener1, fs)
plt.semilogy(f_BPWiener,Pxx_den_BPWiener)
plt.title('Periodogram of signal after Bandpass and Wiener')
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD')
plt.grid()
plt.show()

In [53]:
########################################################################
###  BRUIT DU SIGNAL

##### SIGNAL TEMPOREL 

xmin = 11114*25
xmax = 18511*25

noise = df.iloc[xmin:xmax,1]

plt.figure(figsize = (100,20))
plt.plot(df.index[xmin:xmax], noise, 'b-', linewidth=2, label='noise of signal')
plt.title('Noise of Signal')
plt.xlabel('Time [ms]')
plt.ylabel('Amplitude [µV]')
plt.grid()
plt.axis('tight')
plt.show()

In [54]:
##### PERIODOGRAMME DU BRUIT DU SIGNAL

fs = 25000

plt.figure()
f_Noise, Pxx_den_Noise = signal.periodogram(noise, fs)
plt.semilogy(f_Noise,Pxx_den_Noise)
plt.title('Periodogram of noise of Signal')
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD')
plt.grid()
plt.show()

In [58]:
########################################################################
###  COMPARAISON SIGNAL ORIGINAL // BRUIT DU SIGNAL

##### COMPARAISON DES PERIODOGRAMMES DU SIGNAL ORIGINAL ET DU BRUIT DU SIGNAL

fs = 25000

plt.figure()
plt.semilogy(f_orig,Pxx_den_orig,'b-', linewidth=2, label='original data')
plt.semilogy(f_Noise,Pxx_den_Noise, 'r-', linewidth=2, label='noise of data')
plt.title('Comparison of Periodogram-Original Signal and of Noise of Signal')
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD')
plt.legend()
plt.grid()
plt.show()

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [64]:
########################################################################
###  COMPARAISON SIGNAL ORIGINAL // BRUIT DU SIGNAL // ZONE QUE DES SPIKES

##### COMPARAISON DES PERIODOGRAMMES DU SIGNAL ORIGINAL ET DU BRUIT DU SIGNAL

fs = 25000

#xminspike = 67135*25
#xmaxspike = 87285*25
xminspike = 130826*25
xmaxspike = 131699*25

f_QueSpikes, Pxx_den_QueSpikes = signal.periodogram(df.iloc[xminspike:xmaxspike,1], fs)

plt.figure()
plt.semilogy(f_orig,Pxx_den_orig,'b-', linewidth=2, label='original data')
plt.semilogy(f_QueSpikes,Pxx_den_QueSpikes, 'y-', linewidth=2, label='data where only spikes')
plt.semilogy(f_Noise,Pxx_den_Noise, 'r-', linewidth=2, label='noise of data')
plt.title('Comparison of Periodogram-Original Signal/Noise of Signal/Signal where only Spikes')
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD')
plt.legend()
plt.grid()
plt.show()

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [38]:
########################################################################
###  COMPARAISON SIGNAL ORIGINAL // SIGNAL FILTRÉ PASSE-BANDE

##### SIGNAL TEMPOREL 

plt.figure(figsize = (100,20))
plt.plot(df.index, df.iloc[:,1], 'b-', linewidth=2, label='original data')
plt.plot(df.index, y, 'g-', linewidth=2, label='data after bandpass')
plt.title('Comparison of Original Signal and Signal after Band-Pass')
plt.xlabel('Time [ms]')
plt.ylabel('Amplitude [µV]')
plt.grid()
plt.axis('tight')
plt.legend()
plt.show()

In [69]:
##### COMPARAISON DES PERIODOGRAMMES DU SIGNAL ORIGINAL ET DU SIGNAL APRÈS PASSE-BANDE

fs = 25000

plt.figure()
plt.semilogy(f_orig,Pxx_den_orig,'b-', linewidth=2, label='original data')
plt.semilogy(f_BP,Pxx_den_BP, 'r-', linewidth=2, label='data after Band-Pass')
plt.title('Comparison of Periodogram-Original Signal and Signal after Band-Pass')
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD')
plt.grid()
plt.show()

In [39]:
########################################################################
###  COMPARAISON SIGNAL ORIGINAL // SIGNAL FILTRÉ PASSE-BANDE + WIENER

##### SIGNAL TEMPOREL 

plt.figure(figsize = (100,20))
plt.plot(df.index, df.iloc[:,1], 'b-', linewidth=2, label='original data')
plt.plot(df.index, y_wiener1, 'g-', linewidth=2, label='data after bandpass + Wiener')
plt.title('Comparison of Original Signal and Signal after Band-Pass + Wiener')
plt.xlabel('Time [ms]')
plt.ylabel('Amplitude [µV]')
plt.grid()
plt.axis('tight')
plt.legend()
plt.show()

In [70]:
##### COMPARAISON DES PERIODOGRAMMES DU SIGNAL ORIGINAL ET DU SIGNAL APRÈS PASSE-BANDE+WIENER

fs = 25000

plt.figure()
plt.semilogy(f_orig,Pxx_den_orig,'b-', linewidth=2, label='original data')
plt.semilogy(f_BPWiener,Pxx_den_BPWiener, 'r-', linewidth=2, label='data after Band-Pass+Wiener')
plt.title('Comparison of Periodogram-Original Signal and Signal after Band-Pass+Wiener')
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD')
plt.grid()
plt.legend()
plt.show()

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [46]:
########################################################################
###  COMPARAISON DE TOUS LES SIGNAUX

##### SIGNAL TEMPOREL 

plt.figure(figsize = (100,20))
plt.plot(df.index, df.iloc[:,1], 'b-', linewidth=2, label='original data')
plt.plot(df.index, y, 'g-', linewidth=2, label='data after bandpass')
plt.plot(df.index, y_wiener1, 'r-', linewidth=2, label='data after bandpass + Wiener')
plt.title('Comparison of all the signals')
plt.xlabel('Time [ms]')
plt.ylabel('Amplitude [µV]')
plt.grid()
plt.axis('tight')
plt.legend()
plt.show()

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [78]:
##### COMPARAISON DES PERIODOGRAMMES DE TOUS LES SIGNAUX

fs = 25000

plt.figure()
plt.semilogy(f_orig,Pxx_den_orig,'b-', linewidth=2, label='original data')
plt.semilogy(f_BP,Pxx_den_BP, 'g-', linewidth=2, label='data after Band-Pass')
plt.semilogy(f_BPWiener,Pxx_den_BPWiener, 'r-', linewidth=2, label='data after Band-Pass+Wiener')
plt.title('Comparison of Periodogram of all signals')
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD')
plt.grid()
plt.legend()
plt.show()

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)
